In [ ]:
from datetime import datetime, timedelta
import os
import sys

import numpy as np
import pandas as pd
import laspy
import rasterio

import cars_rasterize.las2tif as las2tif

# Acquisition time
The documentation (https://geoservices.ign.fr/sites/default/files/2023-10/DC_LiDAR_HD_1-0_PTS.pdf Section 2.2.12) states that:

"La valeur du temps (gps_time) du point correspond au nombre de seconde écoulées depuis le
14/09/2011 à 00:00:00 UTC"

### Single laz file

In [ ]:
laz_file = "/path/to/my/point_cloud_1.laz"

In [ ]:
source_time = datetime(2011, 9, 14, 0, 0, 0)

df = pd.DataFrame(columns=["laz_file", "point_count", "min_time", "max_time", "med_time"])
laz = laspy.read(laz_file)
# list(laz_1.point_format.dimension_names)
pc = laz.header.point_count

min_delta = np.min(laz['gps_time'])
max_delta = np.max(laz['gps_time'])
med_delta = np.median(laz['gps_time'])

min_time = source_time + timedelta(seconds=np.floor(min_delta))
max_time = source_time + timedelta(seconds=np.ceil(max_delta))
med_time = source_time + timedelta(seconds=np.ceil(med_delta))

df.loc[0, ["laz_file", "point_count", "min_time", "max_time", "med_time"]] = [laz_file, pc, min_time, max_time, med_time]
df

### Aggregate CSV files

This part reads the .csv files created by compute_laz_date.py to display stats on all .laz

In [ ]:
list_csv = ["/path/to/my/point_cloud_1.laz.csv", "/path/to/my/point_cloud_2.laz.csv"]

In [ ]:
cols = ["laz_file", "point_count", "min_time", "max_time", "med_time", "med_x", "med_y", "min_x", "min_y", "max_x", "max_y"]
df = pd.DataFrame(columns=cols)

for i, csv_file in enumerate(list_csv):
    df_tmp = pd.read_csv(csv_file, index_col=0)
    df.loc[i, cols] = df_tmp.loc[0, cols]
df.set_index("laz_file", inplace=True)
df

In [ ]:
df["med_time"].astype('datetime64[ns]').quantile(0.5, interpolation="midpoint")

# Fuse LAZ

In [ ]:
import warnings

from laspy import CopcReader
from pyproj.crs import CRS

In [ ]:
list_files = [
    "/path/to/my/point_cloud_1.laz",
    "/path/to/my/point_cloud_2.laz"
]
output_laz_path = "/path/to/my/full_point_cloud.laz"

In [ ]:
def check_dimensions_consistency(list_files):
    list_names = [k.name for k in CopcReader.open(list_files[0]).header.point_format.dimensions]
    dims_to_remove = []

    for file in list_files:
        with CopcReader.open(file) as laz_to_add:
            dimensions = [k.name for k in laz_to_add.header.point_format.dimensions]

        dims_to_remove += [k for k in list_names if k not in dimensions]
        dims_to_remove += [k for k in dimensions if k not in list_names]
    return set(dims_to_remove)

In [ ]:
dims_to_remove = check_dimensions_consistency(list_files)
if len(dims_to_remove)>0:
    warnings.warn(f"Dimensions are not consistent accross LAZ files. We will ignore the following dimensions: {dims_to_remove}")

In [ ]:
scales = None
offsets = None
mode = "w"
crs = CRS.from_epsg(2154)  # Lambert93
points=None

crdr = CopcReader.open(list_files[0]).header

for dim in [k for k in dims_to_remove if k in list(crdr.point_format.dimension_names)]:
    crdr.point_format.remove_extra_dimension(dim)
    
new_header = laspy.LasHeader(version=crdr.version, point_format=crdr.point_format) # Creating a header wich is not COPC
new_header.add_crs(crs)
for file in list_files:
    print("Reading", file)
    laz_to_add = laspy.read(file) 
    if scales is None:
        scales = laz_to_add.header.scales
        offsets = laz_to_add.header.offsets
    elif (scales != laz_to_add.header.scales).any() or (offsets != laz_to_add.header.offsets).any():
        raise RuntimeError(f"Scales or offsets are not consistent between laz files\nScales: {scales}  vs {laz_to_add.header.scales}\nOffsets: {offsets} | {laz_to_add.header.offsets}")

    dims_to_remove_file = [k for k in dims_to_remove if k in list(laz_to_add.point_format.dimension_names)]
    if len(dims_to_remove_file) > 0:
        laz_to_add.remove_extra_dims(dims_to_remove_file)

    with laspy.open(output_laz_path, mode=mode, header=new_header) as out:
        if mode=="w":
            out.write_points(laz_to_add.points)
            mode = "a"
        else:
            out.append_points(laz_to_add.points)

# Rasterize

In [ ]:
output_laz_path = "/path/to/my/full_point_cloud.laz"
output_tif_path = "/path/to/my/rasterized_dsm.tif"

In [ ]:
las2tif.main(output_laz_path, output_tif_path, resolution=0.5, radius=3, sigma=0.3)

In [ ]:
# Checking if CRS is correct and nodata is not NaN
flag_write = False
dsm_reader = rasterio.open(output_tif_path)
with rasterio.Env():
    profile = dsm_reader.profile
    if (profile["crs"] is None) or ~(profile["crs"].equals(rasterio.crs.CRS.from_epsg(2154))):
        dsm = dsm_reader.read(1)
        profile.update(crs=rasterio.crs.CRS.from_epsg(2154))
        flag_write = True
    if profile["nodata"] == "nan":
        dsm = dsm_reader.read(1)
        dsm[np.isnan(dsm)] = -32768
        profile.update(nodata=-32768.0)
        flag_write = True
    if flag_write:
        with rasterio.open(output_tif_path, 'w', **profile) as dst:
            dst.write(dsm.astype(profile["dtype"]), 1)